# Setup

In [ ]:
!pip install yt-dlp
!pip install openai
!git clone https://github.com/chisakiShinichirouToshiyuki/commentSupporter.git

from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Manager
from commentSupporter.src.display import replace_watch_comment
from commentSupporter.src.getChat import get_chat
from IPython.display import clear_output
from datetime import datetime, timedelta, timezone


In [ ]:
import getpass
url = input("YoutubeのURLを入力してください: ")
api_key = getpass.getpass("chatGPTを利用する方は、API keyを自己責任で入力してください。: ")
split_url = url.split('https://www.youtube.com/watch?v=')
assert split_url[0] == ''
live_id = split_url[1]

# Promptの設定

In [ ]:
prompt_setting = {
    'model': "gpt-3.5-turbo",
    'messages': [
        {
            "role": "system",
            "content": "あなたはコメント分析の専門家です"
        },
        {
            "role": "user",
            "content": """
                次の一連のコメントから、コメント主の特徴を一言で分析してください。
                「昨日はラーメン食べた」
                「今日はパスタ食べた」
                「明日はうどん食う」
            """
        },
        {
            "role": "assistant",
            "content": "麺好き"
        },
        {"role": "user",
            "content": """
                素晴らしい。
                次の一連のコメントから、コメント主の特徴を一言で分析してください。
                「昨日カレー食べた」
                「今日はカレーグラタン食べた」
                「明日はカレーうどん食う」
            """
        },
        {
            "role": "assistant",
            "content": "カレー好き"
        },
        {
            "role": "user",
            "content":  (
                "素晴らしい。次の一連のコメントから、コメント主の特徴を一言で分析してください。"
            )
        },
    ],
    'temperature': 1 
}


# 中略

In [ ]:
def main(live_id:str):
    with Manager() as manager:
        queue = manager.Queue()
        with ProcessPoolExecutor(max_workers=2) as executor:
            future_1 = executor.submit(replace_watch_comment, queue, live_id, api_key,prompt_setting)
            future_2 = executor.submit(get_chat, live_id)
            while True:
                message = queue.get()
                if message != '':
                    clear_output()
                    print(message)
                elif message == 'DONE':
                    break
                    
            future_1.result()
            future_2.result()



# 下記に、チャットとその分析結果が表示されます
## 初回のチャットが分析されるまで、数十秒かかる場合があります

In [ ]:
main(live_id)

In [ ]:
!pip install ipywidgets

In [ ]:


import ipywidgets as widgets
from IPython.display import display

list1: list[str]
select1: widgets.SelectMultiple
list2:  list[str]
select2: widgets.SelectMultiple
title2  : widgets.Label

# クリックイベントのハンドラー
def update_list(change):
    global list1
    global list2 
    global select1 
    global select2 
    global title2  # 
    if change['new']:
        # "addList"を10回list1に追加
        list1.extend([f"addList{_}" for _ in range(10)])
        list1 = list1[-50:]
        scroll_end_message = [f"もうすぐスクロール端です：{i}" for i in range(5)]
        list_all = scroll_end_message + list1 + scroll_end_message[::-1]
        # select1のoptionsを更新
        select1.options = list_all
        # print('Selected item:', change['new'])
        title2.value = 'Selected item: ' + ', '.join(change['new'])

def analyze(change):
    global list1
    global list2 
    global select1 
    global select2 
    global title2  # 
    if change['new']:
        # "addList"を10回list1に追加
        list1.extend([f"addList{_}" for _ in range(10)])
        list1 = list1[-50:]
        scroll_end_message = [f"もうすぐスクロール端です：{i}" for i in range(5)]
        list_all = scroll_end_message + list1 + scroll_end_message[::-1]
        # select1のoptionsを更新
        select1.options = list_all
        # print('Selected item:', change['new'])
        title2.value = 'Selected item: ' + ', '.join(change['new'])

def set_ui():
    # リストの内容
    global list1
    global list2 
    global select1 
    global select2 
    global title2 
    list1 = [f'item{i+1}' for i in range(50)]
    list2 = [f'item{i+1}' for i in range(50, 81)]

    # スクロール端メッセージの追加
    scroll_end_message = [f"もうすぐスクロール端です：{i}" for i in range(5)] if 30< len(list2) else [] 
    list1_all = scroll_end_message + list1 + scroll_end_message[::-1]
    list2_all = scroll_end_message + list2 + scroll_end_message[::-1]

    # SelectMultiple widgetsの作成
    select1 = widgets.SelectMultiple(
        options=list1_all,
        description='',  # リストタイトルを非表示
        layout=widgets.Layout(width='auto', height='580px', overflow_y='auto')  # スクロールを追加
    )

    select2 = widgets.SelectMultiple(
        options=list2_all,
        layout=widgets.Layout(width='auto', height='500px', overflow_y='auto')  # スクロールを追加
    )


    # イベントハンドラーの追加
    select1.observe(update_list, 'value')

    # タイトルウィジェットの作成
    title2 = widgets.Label(
        value='List 2', 
        layout=widgets.Layout(width='auto')
    )

    # フッターウィジェットの作成
    footer2 = widgets.Label(
        value='Info: This is a footer', 
        layout=widgets.Layout(width='auto')
    )

    # 右側のカラムを作成（タイトル、リスト、フッターを含む）
    right_column = widgets.VBox([title2, select2, footer2], layout=widgets.Layout(width='50%', overflow='auto'))

    # 左側のカラムを作成（リストのみ）
    left_column = widgets.VBox([select1], layout=widgets.Layout(width='50%', overflow='auto'))

    # 2カラムのレイアウトを作成
    two_columns = widgets.HBox([left_column, right_column], layout=widgets.Layout(width='100%', overflow='auto'))

    display(two_columns)

set_ui()